# Prepare Callbacks

In [1]:
import os

In [2]:
%pwd

'f:\\Plant Disease Classification\\Plant Disease Classification\\research\\Team Member Work\\Abdul Jaweed'

In [3]:
os.chdir("../")

In [4]:
%pwd

'f:\\Plant Disease Classification\\Plant Disease Classification\\research\\Team Member Work'

In [5]:
os.chdir("../")
os.chdir("../")
%pwd

'f:\\Plant Disease Classification\\Plant Disease Classification'

In [6]:
# First define artifacts on config.yaml

# config entity

from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path


# Add into config constructor __init__.py -> PrepareCallbacksConfig

In [7]:
from plant.constants import *
from plant.utils import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    # Add on configuration.py
    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config

In [9]:
import os
os.path.dirname("x/y/z.txt")

'x/y'

In [10]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [11]:
import time
timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
f"tb_logs_at_{timestamp}"

'tb_logs_at_2023-06-01-12-36-28'

In [12]:
# Add to components

class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]



In [13]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    
except Exception as e:
    raise e

[2023-06-01 12:36:30,882: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-06-01 12:36:30,888: INFO: common: yaml file: params.yaml loaded successfully]
[2023-06-01 12:36:30,891: INFO: common: created directory at: artifacts]
[2023-06-01 12:36:30,904: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2023-06-01 12:36:30,906: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
